# LSTM for ADL recognition with wrist worn accelerometer

In [1]:
import os

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np

import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print("Using device:", device)

Using device: mps


### Define network structure

In [3]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, n_layers, n_adl):
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_adl = n_adl

        self.lstm = nn.LSTM(input_size, hidden_size, n_layers, batch_first=False)
        self.fc = nn.Linear(hidden_size, n_adl)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        
        # initial hidden- & cell-state
        h0 = torch.zeros(self.n_layers, x.size(1), self.hidden_size).to(device)
        c0 = torch.zeros(self.n_layers, x.size(1), self.hidden_size).to(device)

        # forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # decode hidden state of last time step
        out = out[-1, :, :]
        out = self.fc(out)

        return out

### Create custom dataset

In [4]:
files = os.listdir("../data/train")
print(files)
for f in files:
    name = f.split(".")[0].split("-")[-2]
    print(name)

['Accelerometer-2011-03-24-10-24-02-descend_stairs-f1.txt', 'Accelerometer-2011-03-24-10-24-39-climb_stairs-f1.txt', 'Accelerometer-2011-03-24-10-26-33-comb_hair-f1.txt', 'Accelerometer-2011-03-24-10-25-44-climb_stairs-f1.txt', 'Accelerometer-2011-03-24-10-25-11-descend_stairs-f1.txt', 'Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt', 'Accelerometer-2011-03-24-09-44-34-comb_hair-f1.txt', 'Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt']
descend_stairs
climb_stairs
comb_hair
climb_stairs
descend_stairs
brush_teeth
comb_hair
brush_teeth


In [5]:
data = np.loadtxt("../data/train/" + files[0])
data = torch.from_numpy(data).float().unsqueeze(1)
print(data.shape)

torch.Size([594, 1, 3])


In [6]:
adl = list(set([f.split(".")[0].split("-")[-2] for f in files]))

# sort adl
adl.sort()
print(adl)

['brush_teeth', 'climb_stairs', 'comb_hair', 'descend_stairs']


In [7]:
class ADLDataset(Dataset):

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.files = os.listdir(root_dir)
        self.transform = transform

        self.adl = list(set([f.split(".")[0].split("-")[-2] for f in self.files]))
        self.adl.sort()

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        
        # read file
        file = self.files[idx]
        data = np.loadtxt(self.root_dir + file)

        # convert data to tensor
        data = torch.from_numpy(data).float()
        # add batch dimension to second axis
        #data = data.unsqueeze(1)


        # get label
        label = file.split(".")[0].split("-")[-2]
        label = self.adl.index(label)

        # create one hot vector
        one_hot = torch.zeros(len(self.adl))
        one_hot[label] = 1

        return data, one_hot
        

In [8]:
train_dataset = ADLDataset("../data/train/")

data, one_hot = train_dataset[0]
print(data.shape)
print(one_hot.shape)

torch.Size([594, 3])
torch.Size([4])


### Define hyperparameters

In [10]:
input_size = 3  # freatures: x_acc, y_acc, z_acc
n_adl = len(train_dataset.adl)
print(n_adl)

hidden_size = 128
n_layers = 1

learning_rate = 0.001

4


### Start training loop

In [13]:
def train_one_epoch(model, criterion, optimizer, epoch_index, train_loader, device):

    last_loss = 0.

    for i, (data, label) in enumerate(train_loader):
        data = data.permute(1, 0, 2).to(device)
        
        # remove batch dimension
        label = label.squeeze(0).to(device)

        optimizer.zero_grad()
        outputs = model(data)
        print(outputs.shape)
        loss = criterion(outputs, label)
        loss.backward()

        optimizer.step()

        last_loss = loss.item()
        print("  iteration {} loss: {}".format(i + 1, last_loss))

    return last_loss


In [15]:
# create train_dataset (schon erledigt)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

model = LSTM(input_size, hidden_size, n_layers, n_adl).to(device)

criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_epochs = 10

for epoch in range(n_epochs):
    print("Epoch {}:".format(epoch + 1))
    last_loss = train_one_epoch(model, criterion, optimizer, epoch, train_loader, device)


Epoch 1:
torch.Size([1, 4])


ValueError: Expected input batch_size (1) to match target batch_size (4).